In [2]:
import os

In [3]:
%pwd

'c:\\Users\\Pragyasharava\\Desktop\\leukemia-detection\\research'

In [4]:
os.chdir("../")

In [5]:
%pwd

'c:\\Users\\Pragyasharava\\Desktop\\leukemia-detection'

In [6]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class TrainingConfig:
    root_dir: Path
    trained_model_path: Path
    updated_base_model_path: Path
    training_data: Path
    params_epochs: int
    params_batch_size: int
    params_is_augmentation: bool
    params_image_size: list

In [7]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories
import tensorflow as tf

In [8]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    def get_training_config(self) -> TrainingConfig:
        training = self.config.training
        prepare_base_model = self.config.prepare_base_model
        params = self.params
        training_data = os.path.join(self.config.data_ingestion.unzip_dir, "Blood cell Cancer [ALL]")
        create_directories([
            Path(training.root_dir)
        ])

        training_config = TrainingConfig(
            root_dir=Path(training.root_dir),
            trained_model_path=Path(training.trained_model_path),
            updated_base_model_path=Path(prepare_base_model.updated_base_model_path),
            training_data=Path(training_data),
            params_epochs=params.EPOCHS,
            params_batch_size=params.BATCH_SIZE,
            params_is_augmentation=params.AUGMENTATION,
            params_image_size=params.IMAGE_SIZE
        )

        return training_config

In [10]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
import time
from sklearn.utils.class_weight import compute_class_weight
import numpy as np
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau


In [ ]:
class Training:
    def __init__(self, config: TrainingConfig):
        self.config = config

    def get_base_model(self):
        self.model = tf.keras.models.load_model(
            self.config.updated_base_model_path
        )

    def train_valid_generator(self):

        # Data generator arguments
        datagenerator_kwargs = dict(
            rescale=1./255,
            validation_split=0.20
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        # Validation data generator
        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )

        # Training data generator with augmentation if enabled
        if self.config.params_is_augmentation:
            train_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
                rotation_range=40,
                horizontal_flip=True,
                width_shift_range=0.2,
                height_shift_range=0.2,
                shear_range=0.2,
                zoom_range=0.2,
                **datagenerator_kwargs
            )
        else:
            train_datagenerator = valid_datagenerator

        self.train_generator = train_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="training",
            shuffle=True,
            **dataflow_kwargs
        )

        # Compute class weights to handle class imbalance
        class_weights = compute_class_weight(
            class_weight="balanced",
            classes=np.unique(self.train_generator.classes),
            y=self.train_generator.classes
        )
        self.class_weights = dict(enumerate(class_weights))

    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)

    def train(self):
        # Calculate steps per epoch and validation steps
        self.steps_per_epoch = self.train_generator.samples // self.train_generator.batch_size
        self.validation_steps = self.valid_generator.samples // self.valid_generator.batch_size

        # Define the callbacks for early stopping and reducing learning rate on plateau
        callbacks = [
            EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True),
            ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, min_lr=1e-7, verbose=1)
        ]

        # Train the model with class weights and callbacks
        self.model.fit(
            self.train_generator,
            epochs=self.config.params_epochs,
            steps_per_epoch=self.steps_per_epoch,
            validation_steps=self.validation_steps,
            validation_data=self.valid_generator,
            class_weight=self.class_weights,
            callbacks=callbacks
        )

        # Save the trained model after training
        self.save_model(
            path=self.config.trained_model_path,
            model=self.model
        )


In [12]:
try:
    config = ConfigurationManager()
    training_config = config.get_training_config()
    training = Training(config=training_config)
    training.get_base_model()
    training.train_valid_generator()
    training.train()
    
except Exception as e:
    raise e

[2025-02-22 18:12:11,752: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-02-22 18:12:11,765: INFO: common: yaml file: params.yaml loaded successfully]
[2025-02-22 18:12:11,767: INFO: common: created directory at: artifacts]
[2025-02-22 18:12:11,769: INFO: common: created directory at: artifacts\training]
Found 647 images belonging to 4 classes.
Found 2595 images belonging to 4 classes.
Epoch 1/10
162/162 [==============================] - 543s 3s/step - loss: 1.3041 - accuracy: 0.3746 - val_loss: 1.4161 - val_accuracy: 0.1484 - lr: 1.0000e-05
Epoch 2/10
162/162 [==============================] - 509s 3s/step - loss: 0.9174 - accuracy: 0.6751 - val_loss: 1.5440 - val_accuracy: 0.1375 - lr: 1.0000e-05
Epoch 3/10
162/162 [==============================] - ETA: 0s - loss: 0.6100 - accuracy: 0.8259
Epoch 3: ReduceLROnPlateau reducing learning rate to 4.999999873689376e-06.
162/162 [==============================] - 499s 3s/step - loss: 0.6100 - accuracy: 0.8259 - val